In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import matplotlib.pyplot as plt
from datetime import *
import math

In [2]:
ping_csv = pd.read_csv("../data/measure_ping.csv")
bw_csv = pd.read_csv("../data/measure_bw.csv")
weather_csv = pd.read_csv("../data/measure_weather.csv")

### Weather conditions

In [3]:
weather_array = np.array(weather_csv)
weather_cond = np.unique(weather_array[:,1])
print(weather_cond)

['/staticfiles/images/picto/v2/bc/day/cloudy.png'
 '/staticfiles/images/picto/v2/bc/day/cover.png'
 '/staticfiles/images/picto/v2/bc/day/lightcloudy.png'
 '/staticfiles/images/picto/v2/bc/day/lightrain.png'
 '/staticfiles/images/picto/v2/bc/day/rain.png'
 '/staticfiles/images/picto/v2/bc/day/sun.png'
 '/staticfiles/images/picto/v2/bc/night/cloudy.png'
 '/staticfiles/images/picto/v2/bc/night/cover.png'
 '/staticfiles/images/picto/v2/bc/night/lightcloudy.png'
 '/staticfiles/images/picto/v2/bc/night/lightsun.png'
 '/staticfiles/images/picto/v2/bc/night/mostcloudy.png'
 '/staticfiles/images/picto/v2/bc/night/sun.png']


In [4]:
weather_type = []
for cond in weather_cond:
    weather_type.append(cond.split("/")[7].split(".")[0])
weather_type = np.unique(weather_type)
print(weather_type)
    

['cloudy' 'cover' 'lightcloudy' 'lightrain' 'lightsun' 'mostcloudy' 'rain'
 'sun']


Fusion of the ping measure and the weather measure in one matrix

In [5]:
"""
    Fusion csv1 as a np.array of measure and csv2 the measure of weather
"""
def fusion_with_weather(csv1, csv2):
    fusion = []
    index2 = 0
    for i in range(len(csv1)):
        if(index2 >= len(csv2)):
            return fusion;
        date1 = datetime.strptime(csv1[i][0][0:13], "%Y-%m-%d %H")
        date2 = datetime.strptime(csv2[index2][0][0:13], "%Y-%m-%d %H") + timedelta(hours=1)
        if(date1 == date2):
            # print(np.ndarray.tolist(csv1[i]) + np.ndarray.tolist(csv2[index2][1:]))
            fusion.append(np.ndarray.tolist(csv1[i]) + np.ndarray.tolist(csv2[index2][1:]))
        elif(date1 > date2):
            i -= 1
            index2 +=1
    return fusion

In [6]:
ping_array = np.array(ping_csv)
ping_weather_array = fusion_with_weather(np.array(ping_csv), np.array(weather_csv))

Create a dictionnary { 'weather condition' : '[ping measure]' }

In [7]:
weather_ping_dict = {}
for type in weather_type:
    weather_ping_dict[type] = []

for entry in ping_weather_array:
    weather_ping_dict[entry[6].split("/")[7].split(".")[0]].append(entry[0:5])

In [18]:
"""
    Compute mean pings of a string array of double, ignore NaN
"""
def mean_ping(pings):   
    array = literal_eval(pings)
    measures = np.array(array).astype(np.double)
    
    tot = 0
    non_nan = 0
    for i in range(len(measures)):
        if(not math.isnan(measures[i])):
            tot += measures[i]
            non_nan += 1
    if(non_nan == 0):
        return float('NaN')
    return tot/non_nan

In [20]:
weather_pings = {}
for type in weather_type:
    weather_pings[type] = []
    
for type in weather_ping_dict.keys():
    tot = 0
    non_nan = 0;
    for mes in weather_ping_dict[type]:
        mean = mean_ping(mes[2])
        if(math.isnan(mean)):
            continue
        tot += mean_ping(mes[2])
        non_nan += 1;
    if(non_nan == 0):
        print("ALL measurement are NaN")
        weather_pings[type] = 'NaN'
    weather_pings[type] = tot/non_nan

print(weather_pings)

{'cloudy': 0.1483807408548644, 'cover': 0.1463721185033537, 'lightcloudy': 0.14720876095590732, 'lightrain': 0.14449552542797084, 'lightsun': 0.1447602755273067, 'mostcloudy': 0.1473016696252833, 'rain': 0.14434405683062643, 'sun': 0.14622336385252435}


Sorted weather type acording to mean ping delay

In [25]:
array_tuple = []
for type,mean in weather_pings.items():
    array_tuple.append((type,mean))
array_tuple.sort()
print(array_tuple)

[('cloudy', 0.1483807408548644), ('cover', 0.1463721185033537), ('lightcloudy', 0.14720876095590732), ('lightrain', 0.14449552542797084), ('lightsun', 0.1447602755273067), ('mostcloudy', 0.1473016696252833), ('rain', 0.14434405683062643), ('sun', 0.14622336385252435)]
